In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from category_encoders import TargetEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rajap20/streamlit-example/master/data_final.csv')

In [ ]:
df.head(5)

,Customer_id,Date_disb,pay_type,area_code,pin_code,state,dealer,product_code,tenure,roi,emi,proc_fee,asset_cost,loan_amt,gender,qualification,employ_type,resid_type,age,cibil_score,net_salary,net_irr,fraud
0,0,2/13/2020,ACH,3075,530041,AP,DEALER,SC,0.470588,-0.793282,0.124252,-0.093758,87000,71000,M,OTHERS,SAL,O,0.306122,-0.906781,0.158169,-1.321153,0
1,1,06-02-2020,ADM,3075,532123,AP,DEALER,SC,0.352941,-0.952045,0.083863,-0.572295,61500,51000,F,PG,SAL,O,0.551020,0.163204,1.161336,-1.308269,0
2,2,10-02-2020,ADM,3075,533218,AP,ASC,SC,0.352941,0.318061,0.210079,0.913555,87000,74038,M,SSC,SEP,O,0.530612,NaN,0.910545,0.605033,0
3,3,10-02-2020,ADM,3075,532484,AP,DEALER,SC,0.352941,-1.309262,0.166511,0.649593,84000,69720,M,OTHERS,AGR,O,0.224490,-0.167982,-0.544048,-1.098901,0
4,4,10-02-2020,ADM,3075,530046,AP,DEALER,SC,0.352941,-0.713900,0.092091,-0.572295,61500,52220,F,HSC,SAL,O,0.224490,NaN,-0.343415,-0.986164,0


In [ ]:


df.columns

Index(['Customer_id', 'Date_disb', 'pay_type', 'area_code', 'pin_code',
       'state', 'dealer', 'product_code', 'tenure', 'roi', 'emi', 'proc_fee',
       'asset_cost', 'loan_amt', 'gender', 'qualification', 'employ_type',
       'resid_type', 'age', 'cibil_score', 'net_salary', 'net_irr', 'fraud'],
      dtype='object')

In [ ]:
x_features = ['pay_type', 'area_code','state', 'dealer', 'product_code', 'tenure', 'roi', 'emi', 'proc_fee',
       'asset_cost', 'loan_amt', 'gender', 'qualification', 'employ_type',
       'resid_type', 'age','net_salary', 'net_irr']
#pin_code - Pin Code not included in the model
#cibil_score - CIBIL Score not included in the model
#Customer_id - customer id not included in the model
#Date_disb - date not included in the model

In [ ]:
df = df[x_features + ['fraud']].dropna()
df.shape

(11698, 19)

In [ ]:
cat_features = ['pay_type', 'area_code', 'state', 'dealer', 'product_code','gender', 'qualification', 'employ_type','resid_type']

In [ ]:
df[cat_features] = df[cat_features].astype(object)
df[x_features].info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 11698 entries, 0 to 11701
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   pay_type       11698 non-null  object 
 1   area_code      11698 non-null  object 
 2   state          11698 non-null  object 
 3   dealer         11698 non-null  object 
 4   product_code   11698 non-null  object 
 5   tenure         11698 non-null  float64
 6   roi            11698 non-null  float64
 7   emi            11698 non-null  float64
 8   proc_fee       11698 non-null  float64
 9   asset_cost     11698 non-null  int64  
 10  loan_amt       11698 non-null  int64  
 11  gender         11698 non-null  object 
 12  qualification  11698 non-null  object 
 13  employ_type    11698 non-null  object 
 14  resid_type     11698 non-null  object 
 15  age            11698 non-null  float64
 16  net_salary     11698 non-null  float64
 17  net_irr        11698 non-null  float64
dtypes: flo

In [ ]:
num_features = list(set(x_features) - set(cat_features))
num_features

['age',
 'proc_fee',
 'net_salary',
 'tenure',
 'asset_cost',
 'roi',
 'emi',
 'loan_amt',
 'net_irr']

In [ ]:
target_encoder = TargetEncoder(cols=cat_features, min_samples_leaf=100, smoothing=1000.0)

In [ ]:
target_encoder.fit(df[x_features], df.fraud)

TargetEncoder(cols=['pay_type', 'area_code', 'state', 'dealer', 'product_code',
                    'gender', 'qualification', 'employ_type', 'resid_type'],
              min_samples_leaf=100, smoothing=1000.0)

In [ ]:
df_encoded = target_encoder.transform(df[x_features])

In [ ]:
df_encoded.head(10)

,pay_type,area_code,state,dealer,product_code,tenure,roi,emi,proc_fee,asset_cost,loan_amt,gender,qualification,employ_type,resid_type,age,net_salary,net_irr
0,0.083502,0.045275,0.020545,0.091774,0.088738,0.470588,-0.793282,0.124252,-0.093758,87000,71000,0.069019,0.071796,0.110623,0.06587,0.306122,0.158169,-1.321153
1,0.069421,0.045275,0.020545,0.091774,0.088738,0.352941,-0.952045,0.083863,-0.572295,61500,51000,0.094573,0.079121,0.110623,0.06587,0.551020,1.161336,-1.308269
2,0.069421,0.045275,0.020545,0.054599,0.088738,0.352941,0.318061,0.210079,0.913555,87000,74038,0.069019,0.061191,0.066645,0.06587,0.530612,0.910545,0.605033
3,0.069421,0.045275,0.020545,0.091774,0.088738,0.352941,-1.309262,0.166511,0.649593,84000,69720,0.069019,0.071796,0.041592,0.06587,0.224490,-0.544048,-1.098901
4,0.069421,0.045275,0.020545,0.091774,0.088738,0.352941,-0.713900,0.092091,-0.572295,61500,52220,0.094573,0.070152,0.110623,0.06587,0.224490,-0.343415,-0.986164
5,0.069421,0.045275,0.020545,0.091774,0.088738,0.235294,-0.956014,0.150056,-0.448829,61500,51000,0.094573,0.092105,0.110623,0.06587,0.673469,0.158169,-0.921743
6,0.069421,0.045275,0.020545,0.091774,0.088738,0.352941,-0.436064,0.194185,-0.093758,87000,73000,0.069019,0.061191,0.110623,0.06587,0.142857,-0.343415,-0.126145
7,0.069421,0.045275,0.020545,0.091774,0.088738,0.411765,-1.190190,0.170157,0.705791,86000,70152,0.069019,0.070152,0.066645,0.06587,0.734694,-0.092623,-0.908859
8,0.069421,0.045275,0.020545,0.091774,0.088738,0.549020,0.536361,0.141548,0.743257,84000,70684,0.069019,0.070152,0.041592,0.06587,0.224490,0.408961,0.630801
9,0.069421,0.047452,0.047495,0.091774,0.088738,0.588235,0.754661,0.105553,0.707494,86600,70072,0.094573,0.070152,0.066645,0.06587,0.510204,-0.092623,0.534170


# Feature Selection

### Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_encoded
y = df['fraud']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size = 0.8,
                                                    random_state = 80)

# L1 Selction

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression( penalty = 'l1', C = 1.0, solver = 'liblinear' )

In [ ]:
logreg.fit(X_train, y_train)

LogisticRegression(penalty='l1', solver='liblinear')

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, logreg.predict(X_test)))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      2154
           1       0.00      0.00      0.00       186

    accuracy                           0.92      2340
   macro avg       0.46      0.50      0.48      2340
weighted avg       0.85      0.92      0.88      2340



In [ ]:
l1_selection_df = pd.DataFrame( {"features": x_features,
                                 "coef": np.round(logreg.coef_[0], 2)} )

In [ ]:
l1_selection_df[l1_selection_df.coef == 0.0]

,features,coef
0,pay_type,0.0
4,product_code,0.0
7,emi,0.0
9,asset_cost,-0.0
10,loan_amt,0.0
11,gender,0.0
12,qualification,0.0


### Sequential Feature Selection

In [ ]:
import sklearn
print(sklearn.__version__)

1.0.1


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier

In [ ]:
tree = DecisionTreeClassifier( max_depth = 10 )

In [ ]:
sfs = SequentialFeatureSelector(tree, n_features_to_select=10)

In [ ]:
sfs.fit(X_train, y_train)

SequentialFeatureSelector(estimator=DecisionTreeClassifier(max_depth=10),
                          n_features_to_select=10)

In [ ]:
sfs_features = [feature for feature, selected in zip(x_features, sfs.support_) if selected == True]

In [ ]:
sfs_features

['pay_type',
 'dealer',
 'product_code',
 'tenure',
 'proc_fee',
 'gender',
 'qualification',
 'employ_type',
 'resid_type',
 'net_salary']


### Recursive Feature Elimination (RFE)

- Use a machine learning algorithm as a black box evaluator to find the best subsets of features, and so, they are dependent on the estimator.
- Trains the model iteratively and each time removes the least important feature using the weights of the algorithm as the criterion.
- It is a multivariate method in the sense that it evaluates the relevance of several features considered jointly.
- When used as a ranker, in each iteration, the feature that is removed is added into a stack until all features are tested.
- More than one feature can be removed at a single step for computational efficiency.

In [ ]:
from sklearn.feature_selection import RFE

In [ ]:
rfe_selector = RFE(tree, 
                   n_features_to_select=5, 
                   step=1, 
                   verbose=1)
rfe_selector.fit(X_train, y_train)

Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 features.
Fitting estimator with 8 features.
Fitting estimator with 7 features.
Fitting estimator with 6 features.


RFE(estimator=DecisionTreeClassifier(max_depth=10), n_features_to_select=5,
    verbose=1)

In [ ]:
features_rfe = pd.DataFrame({"features": list(x_features),
                             "rank": rfe_selector.ranking_})
features_rfe.sort_values("rank", ascending=True)

,features,rank
8,proc_fee,1
1,area_code,1
9,asset_cost,1
16,net_salary,1
7,emi,1
17,net_irr,2
15,age,3
10,loan_amt,4
6,roi,5
5,tenure,6


### Embedded Methods

- The embedded methods use statistical criteria e.g. information gain as a filter to select featuresusing a machine learning algorithm and then select the subset of features with the highest significance or importance.

- Embedded methods do not use iterations like RFE.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=100, max_depth = 5, max_features = 0.4)
rf_reg.fit(X_train, y_train)

RandomForestRegressor(max_depth=5, max_features=0.4)

In [ ]:
features_rf_imp = pd.DataFrame({"features": list(x_features),
                                "importance": rf_reg.feature_importances_})
features_rf_imp = features_rf_imp.sort_values("importance", ascending=False).reset_index()
features_rf_imp

,index,features,importance
0,1,area_code,0.376026
1,2,state,0.150598
2,14,resid_type,0.061920
3,17,net_irr,0.049850
4,8,proc_fee,0.045615
5,9,asset_cost,0.044347
6,10,loan_amt,0.044205
7,7,emi,0.040573
8,16,net_salary,0.037381
9,6,roi,0.033743


In [ ]:
features_rf_imp['cumsum'] = features_rf_imp.importance.cumsum()
features_rf_imp

,index,features,importance,cumsum
0,1,area_code,0.376026,0.376026
1,2,state,0.150598,0.526624
2,14,resid_type,0.061920,0.588544
3,17,net_irr,0.049850,0.638394
4,8,proc_fee,0.045615,0.684009
5,9,asset_cost,0.044347,0.728356
6,10,loan_amt,0.044205,0.772561
7,7,emi,0.040573,0.813134
8,16,net_salary,0.037381,0.850515
9,6,roi,0.033743,0.884258


In [ ]:
features_rf_imp["features"].head(12).values

array(['area_code', 'state', 'resid_type', 'net_irr', 'proc_fee',
       'asset_cost', 'loan_amt', 'emi', 'net_salary', 'roi', 'tenure',
       'age'], dtype=object)